In [1]:
'''
ML Lab 03

Prit Kanadiya
211070010

Aim: To implement Candidate Elimination Algorithm on Titanic Dataset.

Theory: The candidate Elimination algorithm finds all hypotheses that match all the given training examples. Unlike in Find-S algorithm, it goes through both negative and positive examples, eliminating any inconsistent hypothesis. Here, we use the idea of version space.

Terminology:
? -> accepts all
None -> rejects all
'''

'\nML Lab 03\n\nPrit Kanadiya\n211070010\n\nAim: To implement Candidate Elimination Algorithm on Titanic Dataset.\n\nTheory: The candidate Elimination algorithm finds all hypotheses that match all the given training examples. Unlike in Find-S algorithm, it goes through both negative and positive examples, eliminating any inconsistent hypothesis.\n\nTerminology:\n? -> accepts all\nNULL -> rejects all\n'

In [2]:
# Import all required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Reading the dataset using read_csv
titanic = pd.read_csv("../data/train.csv")

In [4]:
# Printing first 5 instances of the dataset
titanic.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# Printing last 5 instances of the dataset
titanic.tail()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.00,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.00,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.45,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.00,C148,C
890,891,0,3,"Dooley, Mr. Patrick",male,32.0,0,0,370376,7.75,NaN,Q


In [6]:
# Chceking for null values or unclean instances among the dataset
titanic.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [7]:
# Since we observe null values, we fill first clean our dataset. This is very important step since it allows us to perform analytics on dataset after cleaning it.

# For all null values in age column, we replace them with the mean
age = titanic['Age'].fillna(titanic['Age'].mean())
titanic["Age"] = age

# For all null values in cabin and embarked column, we replace them with the mode
cabin = titanic['Cabin'].fillna(titanic['Cabin'].mode()[0])
titanic["Cabin"] = cabin
embarked = titanic['Embarked'].fillna(titanic['Embarked'].mode()[0])
titanic["Embarked"] = embarked

# We can ignore all the Ticket values since they are unique for each passenger
titanic = titanic.drop(['Ticket'], axis=1)


In [8]:
# Confirming that dataset has no Null values
titanic.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Cabin          0
Embarked       0
dtype: int64

In [9]:
# Our concept it the people who survived the titanic disaster
# Our initial hypotheses is the most specific one
# Concept Learning in Machine Learning can be thought of as a boolean-valued function defined over a large set of training data. 

# We drop Survived column because that is our concept
Survived = titanic["Survived"]
titanic = titanic.drop(["Survived"], axis = 1)

titanic.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Pclass       891 non-null    int64  
 2   Name         891 non-null    object 
 3   Sex          891 non-null    object 
 4   Age          891 non-null    float64
 5   SibSp        891 non-null    int64  
 6   Parch        891 non-null    int64  
 7   Fare         891 non-null    float64
 8   Cabin        891 non-null    object 
 9   Embarked     891 non-null    object 
dtypes: float64(2), int64(4), object(4)
memory usage: 69.7+ KB


In [10]:
# Initializing our hypothesis G and S
S = [None] * (titanic.shape[1])
G = ["?"] * (titanic.shape[1])
print("The most specific hypotesis is: ", S)
print("The most general hypothesis is: ", G)

The most specific hypotesis is:  [None, None, None, None, None, None, None, None, None, None]
The most general hypothesis is:  ['?', '?', '?', '?', '?', '?', '?', '?', '?', '?']


In [ ]:
# Iterating through the dataset and modifying hypothesis accordingly.
# Candidate elimination algorithm goes through both positives and negatives.
# For positive instances, we go from specific to general hypothesis.
# For negative instances, we go from general to specific hypothesis.

for index, row in titanic.iterrows():

    if (Survived[index] == 1):
        # Positive
        for i in range(0, len(S)):
            # Specific to General
            if (S[i] == None):
                S[i] = row[i]
            else:
                if (S[i]!= row[i]):
                    S[i] = '?'
    else:
        # Negative
